# Deep MicroBiome

Aug. 14. 2019
@ Youngwon (youngwon08@gmail.com)

In [1]:
import os
import json
import numpy as np
import pandas as pd
import copy
import logging
import sys

import keras.backend as k
import tensorflow as tf

import matplotlib.pyplot as plt
%matplotlib inline

os.environ['CUDA_VISIBLE_DEVICES']=''

/DATA/home/muha/.local/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/usr/local/lib/python3.5/dist-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.5/dist-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.5/dist-packages/tensorflow/python/framework/dtyp

In [2]:
from deepbiome.deepbiome import *

In [3]:
if not tf.__version__.startswith('2'): 
    config = tf.ConfigProto(gpu_options=tf.GPUOptions(allow_growth=True))
    k.set_session(tf.Session(config=config))

## Pick Models

In [4]:
save = False
kfold=1000
# kfold=20
network_model_keys = ['optimizer','lr','decay']
architecture_keys = ['weight_decay', 'weight_l1_penalty', #'weight_l2_penalty',
                     'tree_thrd', 'weight_initial',
                     'batch_normalization','drop_out']
network_training_keys = ['batch_size','epochs']

logging.basicConfig(format = '[%(name)-8s|%(levelname)s|%(filename)s:%(lineno)s] %(message)s',
                    level=logging.DEBUG)
log = logging.getLogger()

In [5]:
#######################################################################
filenames = 'simulation_s0.Rmd'
models = [
          'simulation_s0/simulation_s0_deep',
          'simulation_s0/simulation_s0_deep_l1',
          'simulation_s0/simulation_s0_deepbiome',
         ]

models_aka = [
          'DNN',
          'DNN+$\ell_1$',
          'DeepBiome',
         ]
num_classes = 0
########################################################################
# filenames = 'simulation_s1.Rmd'
# models = [
#           'simulation_s1/simulation_s1_deep',
#           'simulation_s1/simulation_s1_deep_l1',
#           'simulation_s1/simulation_s1_deepbiome',
#          ]

# models_aka = [
#           'DNN',
#           'DNN+$\ell_1$',
#           'DeepBiome',
#          ]
# num_classes = 0
# ########################################################################
# filenames = 'simulation_s2.Rmd'
# models = [
#           'simulation_s2/simulation_s2_deep',
#           'simulation_s2/simulation_s2_deep_l1',
#           'simulation_s2/simulation_s2_deepbiome',
#          ]

# models_aka = [
#           'DNN',
#           'DNN+$\ell_1$',
#           'DeepBiome',
#          ]
# num_classes = 1
# #######################################################################
# filenames = 'simulation_s3.Rmd'
# models = [
#           'simulation_s3/simulation_s3_deep',
#           'simulation_s3/simulation_s3_deep_l1',
#           'simulation_s3/simulation_s3_deepbiome',
#          ]

# models_aka = [
#           'DNN',
#           'DNN+$\ell_1$',
#           'DeepBiome',
#          ]
# num_classes = 3
# # ########################################################################
# filenames = 'simulation_s4.Rmd'
# models = [
#           'simulation_s4/simulation_s4_deep',
#           'simulation_s4/simulation_s4_deep_l1',
#           'simulation_s4/simulation_s4_deepbiome',
#          ]

# models_aka = [
#           'DNN',
#           'DNN+$\ell_1$',
#           'DeepBiome',
#          ]
# num_classes = 0
########################################################################
# filenames = 'simulation_s5.Rmd'
# models = [
#           'simulation_s5/simulation_s5_deep',
#           'simulation_s5/simulation_s5_deep_l1',
#           'simulation_s5/simulation_s5_deepbiome',
#          ]

# models_aka = [
#           'DNN',
#           'DNN+$\ell_1$',
#           'DeepBiome',
#          ]
# num_classes = 0
########################################################################

In [6]:
model_network_info = {}
model_path_info = {}
for model_path in models:
    config_data = configuration.Configurator('%s/config/path_info.cfg' % model_path, log, verbose=False)
    config_data.set_config_map(config_data.get_section_map())
    config_network = configuration.Configurator('%s/config/network_info.cfg' % model_path, log, verbose=False)
    config_network.set_config_map(config_network.get_section_map())

    model_path_info[model_path] = config_data.get_config_map()
    model_network_info[model_path] = config_network.get_config_map()
    
if num_classes == 0: y_names = ['loss','correlation_coefficient']
elif num_classes==1: y_names = ['loss','binary_accuracy','sensitivity','specificity','gmeasure', 'auc']
else: y_names=['loss','categorical_accuracy','precision','recall','f1', 'auc']

if num_classes == 0: measure_index = np.array([0,1])
elif num_classes==1: measure_index = np.array([2,3,4,1,5])
else: measure_index = np.array([1,2,3,4,5])

## Accuracy

In [7]:
results = []
# log.info('%20s & %s' % ('model', '& '.join(['%s ' % name for name in np.array(y_names)[[measure_index]]])))
# print('%10s & %s \\\\\ \hline' % ('model', '& '.join(['%7s &   (sd) ' % name for name in np.array(y_names)[[measure_index]]])))
# for model, aka in zip(models, models_aka):
#     evaluation = np.load('%s/eval.npy' % model)
#     log.info('%20s: %s' % (aka, ''.join(['%10.4f (%10.4f)'%(mean, std) for mean, std in zip(np.mean(evaluation, axis=0),np.std(evaluation, axis=0))])))
#     results.append(np.vstack([np.mean(evaluation, axis=0),np.std(evaluation, axis=0)]).transpose())
for model, aka in zip(models, models_aka):
    train_evaluation = np.load('%s/train_eval.npy' % model)[:,measure_index]
    train_res = '&'.join(['%7.3f & %7.3f'%(mean, std) for mean, std in zip(np.nanmean(train_evaluation, axis=0),np.nanstd(train_evaluation, axis=0))])
    test_evaluation = np.load('%s/test_eval.npy' % model)[:,measure_index]
    test_res = '&'.join(['%7.3f & %7.3f'%(mean, std) for mean, std in zip(np.nanmean(test_evaluation, axis=0),np.nanstd(test_evaluation, axis=0))])
#     log.info('%s & %s & %s \\\\' % (aka, train_res, test_res))
    print('%10s & %s & %s \\\\' % (aka, test_res, train_res))
#     results.append(np.vstack([np.mean(evaluation, axis=0),np.std(evaluation, axis=0)]).transpose())

       DNN &   0.076 &   0.040&  0.874 &   0.077 &   0.032 &   0.034&  0.947 &   0.067 \\
DNN+$\ell_1$ &   0.075 &   0.040&  0.875 &   0.073 &   0.034 &   0.039&  0.945 &   0.068 \\
 DeepBiome &   0.071 &   0.036&  0.882 &   0.069 &   0.043 &   0.034&  0.929 &   0.061 \\


# Weight estimation of DeepBiom

We identify the largest weight estimatio of neurons in two hidden layers; by doing this, we can identify the strongest phylogenetic connections. We compute the True Positive Rate (``TPR``, sensitivity), True Negative Rate (``TNR``, specificity), and their geometric mean (i.e., ``g-Measure``). The false discovery rate (FDR) would be ``FDR = 1-TPR`` in our case.

## DNN + $\ell_1$

In [8]:
num=1
model_path = models[num]
model_aka = models_aka[num]

config_data = configuration.Configurator('%s/config/path_info.cfg' % model_path, log, verbose=False)
config_data.set_config_map(config_data.get_section_map())
config_network = configuration.Configurator('%s/config/network_info.cfg' % model_path, log, verbose=False)
config_network.set_config_map(config_network.get_section_map())

path_info = config_data.get_config_map()
network_info = config_network.get_config_map()

path_info['data_info']['data_path'] = '/'.join(path_info['data_info']['data_path'].split('/')[2:])
path_info['data_info']['tree_info_path'] = '/'.join(path_info['data_info']['tree_info_path'].split('/')[2:])
try: path_info['data_info']['count_list_path'] = '/'.join(path_info['data_info']['count_list_path'].split('/')[2:])
except: pass
try: path_info['data_info']['count_path'] = '/'.join(path_info['data_info']['count_path'].split('/')[2:])
except: pass
path_info['data_info']['idx_path'] = '/'.join(path_info['data_info']['idx_path'].split('/')[2:])

path_info['model_info']['model_dir'] = './%s/%s'%(model_path,path_info['model_info']['model_dir'])

log.info('%22s : %s' % ('model', model_path))
log.info('%22s : %s' % ('model_aka', model_aka))
for k in architecture_keys:
    log.info('%22s : %s' % (k, network_info['architecture_info'].get(k, None)))
for k in network_model_keys:
    log.info('%22s : %s' % (k, network_info['model_info'].get(k, None)))
for k in network_training_keys:
    log.info('%22s : %s' % (k, network_info['training_info'].get(k, None)))

[root    |INFO|<ipython-input-8-1bf6d82da027>:23]                  model : simulation_s0/simulation_s0_deep_l1
[root    |INFO|<ipython-input-8-1bf6d82da027>:24]              model_aka : DNN+$\ell_1$
[root    |INFO|<ipython-input-8-1bf6d82da027>:26]           weight_decay : None
[root    |INFO|<ipython-input-8-1bf6d82da027>:26]      weight_l1_penalty : 0.01
[root    |INFO|<ipython-input-8-1bf6d82da027>:26]              tree_thrd : None
[root    |INFO|<ipython-input-8-1bf6d82da027>:26]         weight_initial : glorot_uniform
[root    |INFO|<ipython-input-8-1bf6d82da027>:26]    batch_normalization : False
[root    |INFO|<ipython-input-8-1bf6d82da027>:26]               drop_out : 0
[root    |INFO|<ipython-input-8-1bf6d82da027>:28]              optimizer : adam
[root    |INFO|<ipython-input-8-1bf6d82da027>:28]                     lr : 0.01
[root    |INFO|<ipython-input-8-1bf6d82da027>:28]                  decay : 0.0001
[root    |INFO|<ipython-input-8-1bf6d82da027>:30]             batch_siz

In [9]:
tw_1 = np.load('%s/tw_1.npy' % path_info['data_info']['data_path'])
tw_2 = np.load('%s/tw_2.npy' % path_info['data_info']['data_path'])
tw_3 = np.load('%s/tw_3.npy' % path_info['data_info']['data_path'])
tw_4 = np.load('%s/tw_4.npy' % path_info['data_info']['data_path'])
true_tree_weight_list = []
for fold in range(kfold):
    true_tree_weight_list.append(np.array([tw_1[fold],tw_2[fold],tw_3[fold],tw_4[fold]]))
# true_tree_weight_list = np.array(true_tree_weight_list)
# np.save('../deepbiome/tests/data/true_weight_list.npy', true_tree_weight_list)

In [10]:
trained_weight_path_list = ['%s/weight/weight_%d.h5' % (path_info['model_info']['model_dir'], i) for i in range(kfold)]

In [11]:
summary = deepbiome_taxa_selection_performance(log, network_info, path_info, num_classes, true_tree_weight_list, trained_weight_path_list)
summary.iloc[0,0] = model_aka

[root    |INFO|readers.py:58] -----------------------------------------------------------------------
[root    |INFO|readers.py:59] Construct Dataset
[root    |INFO|readers.py:60] -----------------------------------------------------------------------
[root    |INFO|readers.py:61] Load data


Instructions for updating:
Colocations handled automatically by placer.


[tensorflow|WARNING|deprecation.py:328] From /usr/local/lib/python3.5/dist-packages/tensorflow/python/ops/resource_variable_ops.py:432: colocate_with (from tensorflow.python.framework.ops) is deprecated and will be removed in a future version.
Instructions for updating:
Colocations handled automatically by placer.
[root    |INFO|build_network.py:84] Load trained model weight at ./simulation_s0/simulation_s0_deep_l1/.//weight/weight_0.h5 
[root    |INFO|build_network.py:84] Load trained model weight at ./simulation_s0/simulation_s0_deep_l1/.//weight/weight_1.h5 
[root    |INFO|build_network.py:84] Load trained model weight at ./simulation_s0/simulation_s0_deep_l1/.//weight/weight_2.h5 
[root    |INFO|build_network.py:84] Load trained model weight at ./simulation_s0/simulation_s0_deep_l1/.//weight/weight_3.h5 
[root    |INFO|build_network.py:84] Load trained model weight at ./simulation_s0/simulation_s0_deep_l1/.//weight/weight_4.h5 
[root    |INFO|build_network.py:84] Load trained model

[root    |INFO|build_network.py:84] Load trained model weight at ./simulation_s0/simulation_s0_deep_l1/.//weight/weight_63.h5 
[root    |INFO|build_network.py:84] Load trained model weight at ./simulation_s0/simulation_s0_deep_l1/.//weight/weight_64.h5 
[root    |INFO|build_network.py:84] Load trained model weight at ./simulation_s0/simulation_s0_deep_l1/.//weight/weight_65.h5 
[root    |INFO|build_network.py:84] Load trained model weight at ./simulation_s0/simulation_s0_deep_l1/.//weight/weight_66.h5 
[root    |INFO|build_network.py:84] Load trained model weight at ./simulation_s0/simulation_s0_deep_l1/.//weight/weight_67.h5 
[root    |INFO|build_network.py:84] Load trained model weight at ./simulation_s0/simulation_s0_deep_l1/.//weight/weight_68.h5 
[root    |INFO|build_network.py:84] Load trained model weight at ./simulation_s0/simulation_s0_deep_l1/.//weight/weight_69.h5 
[root    |INFO|build_network.py:84] Load trained model weight at ./simulation_s0/simulation_s0_deep_l1/.//weigh

[root    |INFO|build_network.py:84] Load trained model weight at ./simulation_s0/simulation_s0_deep_l1/.//weight/weight_128.h5 
[root    |INFO|build_network.py:84] Load trained model weight at ./simulation_s0/simulation_s0_deep_l1/.//weight/weight_129.h5 
[root    |INFO|build_network.py:84] Load trained model weight at ./simulation_s0/simulation_s0_deep_l1/.//weight/weight_130.h5 
[root    |INFO|build_network.py:84] Load trained model weight at ./simulation_s0/simulation_s0_deep_l1/.//weight/weight_131.h5 
[root    |INFO|build_network.py:84] Load trained model weight at ./simulation_s0/simulation_s0_deep_l1/.//weight/weight_132.h5 
[root    |INFO|build_network.py:84] Load trained model weight at ./simulation_s0/simulation_s0_deep_l1/.//weight/weight_133.h5 
[root    |INFO|build_network.py:84] Load trained model weight at ./simulation_s0/simulation_s0_deep_l1/.//weight/weight_134.h5 
[root    |INFO|build_network.py:84] Load trained model weight at ./simulation_s0/simulation_s0_deep_l1/.

...

[root    |INFO|build_network.py:84] Load trained model weight at ./simulation_s0/simulation_s0_deep_l1/.//weight/weight_713.h5 
[root    |INFO|build_network.py:84] Load trained model weight at ./simulation_s0/simulation_s0_deep_l1/.//weight/weight_714.h5 
[root    |INFO|build_network.py:84] Load trained model weight at ./simulation_s0/simulation_s0_deep_l1/.//weight/weight_715.h5 
[root    |INFO|build_network.py:84] Load trained model weight at ./simulation_s0/simulation_s0_deep_l1/.//weight/weight_716.h5 
[root    |INFO|build_network.py:84] Load trained model weight at ./simulation_s0/simulation_s0_deep_l1/.//weight/weight_717.h5 
[root    |INFO|build_network.py:84] Load trained model weight at ./simulation_s0/simulation_s0_deep_l1/.//weight/weight_718.h5 
[root    |INFO|build_network.py:84] Load trained model weight at ./simulation_s0/simulation_s0_deep_l1/.//weight/weight_719.h5 
[root    |INFO|build_network.py:84] Load trained model weight at ./simulation_s0/simulation_s0_deep_l1/.

[root    |INFO|build_network.py:84] Load trained model weight at ./simulation_s0/simulation_s0_deep_l1/.//weight/weight_778.h5 
[root    |INFO|build_network.py:84] Load trained model weight at ./simulation_s0/simulation_s0_deep_l1/.//weight/weight_779.h5 
[root    |INFO|build_network.py:84] Load trained model weight at ./simulation_s0/simulation_s0_deep_l1/.//weight/weight_780.h5 
[root    |INFO|build_network.py:84] Load trained model weight at ./simulation_s0/simulation_s0_deep_l1/.//weight/weight_781.h5 
[root    |INFO|build_network.py:84] Load trained model weight at ./simulation_s0/simulation_s0_deep_l1/.//weight/weight_782.h5 
[root    |INFO|build_network.py:84] Load trained model weight at ./simulation_s0/simulation_s0_deep_l1/.//weight/weight_783.h5 
[root    |INFO|build_network.py:84] Load trained model weight at ./simulation_s0/simulation_s0_deep_l1/.//weight/weight_784.h5 
[root    |INFO|build_network.py:84] Load trained model weight at ./simulation_s0/simulation_s0_deep_l1/.

[root    |INFO|build_network.py:84] Load trained model weight at ./simulation_s0/simulation_s0_deep_l1/.//weight/weight_843.h5 
[root    |INFO|build_network.py:84] Load trained model weight at ./simulation_s0/simulation_s0_deep_l1/.//weight/weight_844.h5 
[root    |INFO|build_network.py:84] Load trained model weight at ./simulation_s0/simulation_s0_deep_l1/.//weight/weight_845.h5 
[root    |INFO|build_network.py:84] Load trained model weight at ./simulation_s0/simulation_s0_deep_l1/.//weight/weight_846.h5 
[root    |INFO|build_network.py:84] Load trained model weight at ./simulation_s0/simulation_s0_deep_l1/.//weight/weight_847.h5 
[root    |INFO|build_network.py:84] Load trained model weight at ./simulation_s0/simulation_s0_deep_l1/.//weight/weight_848.h5 
[root    |INFO|build_network.py:84] Load trained model weight at ./simulation_s0/simulation_s0_deep_l1/.//weight/weight_849.h5 
[root    |INFO|build_network.py:84] Load trained model weight at ./simulation_s0/simulation_s0_deep_l1/.

[root    |INFO|build_network.py:84] Load trained model weight at ./simulation_s0/simulation_s0_deep_l1/.//weight/weight_908.h5 
[root    |INFO|build_network.py:84] Load trained model weight at ./simulation_s0/simulation_s0_deep_l1/.//weight/weight_909.h5 
[root    |INFO|build_network.py:84] Load trained model weight at ./simulation_s0/simulation_s0_deep_l1/.//weight/weight_910.h5 
[root    |INFO|build_network.py:84] Load trained model weight at ./simulation_s0/simulation_s0_deep_l1/.//weight/weight_911.h5 
[root    |INFO|build_network.py:84] Load trained model weight at ./simulation_s0/simulation_s0_deep_l1/.//weight/weight_912.h5 
[root    |INFO|build_network.py:84] Load trained model weight at ./simulation_s0/simulation_s0_deep_l1/.//weight/weight_913.h5 
[root    |INFO|build_network.py:84] Load trained model weight at ./simulation_s0/simulation_s0_deep_l1/.//weight/weight_914.h5 
[root    |INFO|build_network.py:84] Load trained model weight at ./simulation_s0/simulation_s0_deep_l1/.

[root    |INFO|build_network.py:84] Load trained model weight at ./simulation_s0/simulation_s0_deep_l1/.//weight/weight_973.h5 
[root    |INFO|build_network.py:84] Load trained model weight at ./simulation_s0/simulation_s0_deep_l1/.//weight/weight_974.h5 
[root    |INFO|build_network.py:84] Load trained model weight at ./simulation_s0/simulation_s0_deep_l1/.//weight/weight_975.h5 
[root    |INFO|build_network.py:84] Load trained model weight at ./simulation_s0/simulation_s0_deep_l1/.//weight/weight_976.h5 
[root    |INFO|build_network.py:84] Load trained model weight at ./simulation_s0/simulation_s0_deep_l1/.//weight/weight_977.h5 
[root    |INFO|build_network.py:84] Load trained model weight at ./simulation_s0/simulation_s0_deep_l1/.//weight/weight_978.h5 
[root    |INFO|build_network.py:84] Load trained model weight at ./simulation_s0/simulation_s0_deep_l1/.//weight/weight_979.h5 
[root    |INFO|build_network.py:84] Load trained model weight at ./simulation_s0/simulation_s0_deep_l1/.

In [12]:
summary

,Model,PhyloTree,No. true taxa,No. total taxa,Sensitivity_mean,Sensitivity_std,Specificity_mean,Specificity_std,Gmeasure_mean,Gmeasure_std,Accuracy_mean,Accuracy_std
0,DNN+$\ell_1$,Genus,31,48,0.967355,0.032126,0.034048,0.006010,0.180710,0.016248,0.049117,0.005893
1,,Family,23,40,0.970043,0.036123,0.031497,0.006035,0.173975,0.017210,0.054961,0.005993
2,,Order,9,23,0.972222,0.055277,0.025866,0.008119,0.156419,0.025851,0.047650,0.007998
3,,Class,7,17,0.978000,0.055748,0.021247,0.011870,0.136186,0.047452,0.065020,0.011683


In [13]:
print('%7s & %7s & %12s & %s' % ('Model', 'PhyloTree', 'True (Total)', ' & '.join(summary.columns[4:])))
print('---------------------------------------------------------------------------------------------------------------')
for i in range(summary.shape[0]):
    print('%10s & %7s & %7d (%d) & ' % tuple(summary.iloc[i,:4]) + ' &'.join(['%6.3f' % val for val in summary.iloc[i,4:]]) + ' \\\\')
    
# if save: 
#     # filenametexa = '.'.join(["%s_select_texa_1" % filename.split('.')[0], filename.split('.')[1]])
#     colname = ['Tree','True (Total)','Selected','Sensitivity','Specificity','gMeasure','Accuracy']
#     with open('%s/%s' % (analysis_dir, filename), mode='a') as f:
#     #     f.write('---\ntitle: "%s texa selection ver.1"\noutput: html_document\n---\n\n' % filename.split('.')[0])
#         f.write('\n## Texa Selection Preformance (ver 1): %s\n\n' % model_aka)
#         f.write('| %s |\n' % ('|'.join([v for v in colname])))
#         f.write('|'+'---|'*len(colname)+'\n')
#         for value in values:
#             f.write('| %s |\n' % ('|'.join(value)))

  Model & PhyloTree & True (Total) & Sensitivity_mean & Sensitivity_std & Specificity_mean & Specificity_std & Gmeasure_mean & Gmeasure_std & Accuracy_mean & Accuracy_std
---------------------------------------------------------------------------------------------------------------
DNN+$\ell_1$ &   Genus &      31 (48) &  0.967 & 0.032 & 0.034 & 0.006 & 0.181 & 0.016 & 0.049 & 0.006 \\
           &  Family &      23 (40) &  0.970 & 0.036 & 0.031 & 0.006 & 0.174 & 0.017 & 0.055 & 0.006 \\
           &   Order &       9 (23) &  0.972 & 0.055 & 0.026 & 0.008 & 0.156 & 0.026 & 0.048 & 0.008 \\
           &   Class &       7 (17) &  0.978 & 0.056 & 0.021 & 0.012 & 0.136 & 0.047 & 0.065 & 0.012 \\


## DeepBiome

In [14]:
num=2
model_path = models[num]
model_aka = models_aka[num]

config_data = configuration.Configurator('%s/config/path_info.cfg' % model_path, log, verbose=False)
config_data.set_config_map(config_data.get_section_map())
config_network = configuration.Configurator('%s/config/network_info.cfg' % model_path, log, verbose=False)
config_network.set_config_map(config_network.get_section_map())

path_info = config_data.get_config_map()
network_info = config_network.get_config_map()

path_info['data_info']['data_path'] = '/'.join(path_info['data_info']['data_path'].split('/')[2:])
path_info['data_info']['tree_info_path'] = '/'.join(path_info['data_info']['tree_info_path'].split('/')[2:])
try: path_info['data_info']['count_list_path'] = '/'.join(path_info['data_info']['count_list_path'].split('/')[2:])
except: pass
try: path_info['data_info']['count_path'] = '/'.join(path_info['data_info']['count_path'].split('/')[2:])
except: pass
path_info['data_info']['idx_path'] = '/'.join(path_info['data_info']['idx_path'].split('/')[2:])

path_info['model_info']['model_dir'] = './%s/%s'%(model_path,path_info['model_info']['model_dir'])

log.info('%22s : %s' % ('model', model_path))
log.info('%22s : %s' % ('model_aka', model_aka))
for k in architecture_keys:
    log.info('%22s : %s' % (k, network_info['architecture_info'].get(k, None)))
for k in network_model_keys:
    log.info('%22s : %s' % (k, network_info['model_info'].get(k, None)))
for k in network_training_keys:
    log.info('%22s : %s' % (k, network_info['training_info'].get(k, None)))

[root    |INFO|<ipython-input-14-6e33d2ded9d6>:23]                  model : simulation_s0/simulation_s0_deepbiome
[root    |INFO|<ipython-input-14-6e33d2ded9d6>:24]              model_aka : DeepBiome
[root    |INFO|<ipython-input-14-6e33d2ded9d6>:26]           weight_decay : phylogenetic_tree
[root    |INFO|<ipython-input-14-6e33d2ded9d6>:26]      weight_l1_penalty : None
[root    |INFO|<ipython-input-14-6e33d2ded9d6>:26]              tree_thrd : None
[root    |INFO|<ipython-input-14-6e33d2ded9d6>:26]         weight_initial : glorot_uniform
[root    |INFO|<ipython-input-14-6e33d2ded9d6>:26]    batch_normalization : False
[root    |INFO|<ipython-input-14-6e33d2ded9d6>:26]               drop_out : 0
[root    |INFO|<ipython-input-14-6e33d2ded9d6>:28]              optimizer : adam
[root    |INFO|<ipython-input-14-6e33d2ded9d6>:28]                     lr : 0.01
[root    |INFO|<ipython-input-14-6e33d2ded9d6>:28]                  decay : 0.0001
[root    |INFO|<ipython-input-14-6e33d2ded9d6>:3

### Performance

In [15]:
tw_1 = np.load('%s/tw_1.npy' % path_info['data_info']['data_path'])
tw_2 = np.load('%s/tw_2.npy' % path_info['data_info']['data_path'])
tw_3 = np.load('%s/tw_3.npy' % path_info['data_info']['data_path'])
tw_4 = np.load('%s/tw_4.npy' % path_info['data_info']['data_path'])
true_tree_weight_list = []
for fold in range(kfold):
    true_tree_weight_list.append(np.array([tw_1[fold],tw_2[fold],tw_3[fold],tw_4[fold]]))
# true_tree_weight_list = np.array(true_tree_weight_list)
# np.save('../deepbiome/tests/data/true_weight_list.npy', true_tree_weight_list)

In [16]:
trained_weight_path_list = ['%s/weight/weight_%d.h5' % (path_info['model_info']['model_dir'], i) for i in range(kfold)]

In [17]:
summary = deepbiome_taxa_selection_performance(log, network_info, path_info, num_classes, true_tree_weight_list, trained_weight_path_list)
summary.iloc[0,0] = model_aka

[root    |INFO|readers.py:58] -----------------------------------------------------------------------
[root    |INFO|readers.py:59] Construct Dataset
[root    |INFO|readers.py:60] -----------------------------------------------------------------------
[root    |INFO|readers.py:61] Load data
[root    |INFO|build_network.py:84] Load trained model weight at ./simulation_s0/simulation_s0_deepbiome/.//weight/weight_0.h5 
[root    |INFO|build_network.py:84] Load trained model weight at ./simulation_s0/simulation_s0_deepbiome/.//weight/weight_1.h5 
[root    |INFO|build_network.py:84] Load trained model weight at ./simulation_s0/simulation_s0_deepbiome/.//weight/weight_2.h5 
[root    |INFO|build_network.py:84] Load trained model weight at ./simulation_s0/simulation_s0_deepbiome/.//weight/weight_3.h5 
[root    |INFO|build_network.py:84] Load trained model weight at ./simulation_s0/simulation_s0_deepbiome/.//weight/weight_4.h5 
[root    |INFO|build_network.py:84] Load trained model weight at ./s

[root    |INFO|build_network.py:84] Load trained model weight at ./simulation_s0/simulation_s0_deepbiome/.//weight/weight_62.h5 
[root    |INFO|build_network.py:84] Load trained model weight at ./simulation_s0/simulation_s0_deepbiome/.//weight/weight_63.h5 
[root    |INFO|build_network.py:84] Load trained model weight at ./simulation_s0/simulation_s0_deepbiome/.//weight/weight_64.h5 
[root    |INFO|build_network.py:84] Load trained model weight at ./simulation_s0/simulation_s0_deepbiome/.//weight/weight_65.h5 
[root    |INFO|build_network.py:84] Load trained model weight at ./simulation_s0/simulation_s0_deepbiome/.//weight/weight_66.h5 
[root    |INFO|build_network.py:84] Load trained model weight at ./simulation_s0/simulation_s0_deepbiome/.//weight/weight_67.h5 
[root    |INFO|build_network.py:84] Load trained model weight at ./simulation_s0/simulation_s0_deepbiome/.//weight/weight_68.h5 
[root    |INFO|build_network.py:84] Load trained model weight at ./simulation_s0/simulation_s0_de

[root    |INFO|build_network.py:84] Load trained model weight at ./simulation_s0/simulation_s0_deepbiome/.//weight/weight_126.h5 
[root    |INFO|build_network.py:84] Load trained model weight at ./simulation_s0/simulation_s0_deepbiome/.//weight/weight_127.h5 
[root    |INFO|build_network.py:84] Load trained model weight at ./simulation_s0/simulation_s0_deepbiome/.//weight/weight_128.h5 
[root    |INFO|build_network.py:84] Load trained model weight at ./simulation_s0/simulation_s0_deepbiome/.//weight/weight_129.h5 
[root    |INFO|build_network.py:84] Load trained model weight at ./simulation_s0/simulation_s0_deepbiome/.//weight/weight_130.h5 
[root    |INFO|build_network.py:84] Load trained model weight at ./simulation_s0/simulation_s0_deepbiome/.//weight/weight_131.h5 
[root    |INFO|build_network.py:84] Load trained model weight at ./simulation_s0/simulation_s0_deepbiome/.//weight/weight_132.h5 
[root    |INFO|build_network.py:84] Load trained model weight at ./simulation_s0/simulatio

[root    |INFO|build_network.py:84] Load trained model weight at ./simulation_s0/simulation_s0_deepbiome/.//weight/weight_190.h5 
[root    |INFO|build_network.py:84] Load trained model weight at ./simulation_s0/simulation_s0_deepbiome/.//weight/weight_191.h5 
[root    |INFO|build_network.py:84] Load trained model weight at ./simulation_s0/simulation_s0_deepbiome/.//weight/weight_192.h5 
[root    |INFO|build_network.py:84] Load trained model weight at ./simulation_s0/simulation_s0_deepbiome/.//weight/weight_193.h5 
[root    |INFO|build_network.py:84] Load trained model weight at ./simulation_s0/simulation_s0_deepbiome/.//weight/weight_194.h5 
[root    |INFO|build_network.py:84] Load trained model weight at ./simulation_s0/simulation_s0_deepbiome/.//weight/weight_195.h5 
[root    |INFO|build_network.py:84] Load trained model weight at ./simulation_s0/simulation_s0_deepbiome/.//weight/weight_196.h5 
[root    |INFO|build_network.py:84] Load trained model weight at ./simulation_s0/simulatio

[root    |INFO|build_network.py:84] Load trained model weight at ./simulation_s0/simulation_s0_deepbiome/.//weight/weight_254.h5 
[root    |INFO|build_network.py:84] Load trained model weight at ./simulation_s0/simulation_s0_deepbiome/.//weight/weight_255.h5 
[root    |INFO|build_network.py:84] Load trained model weight at ./simulation_s0/simulation_s0_deepbiome/.//weight/weight_256.h5 
[root    |INFO|build_network.py:84] Load trained model weight at ./simulation_s0/simulation_s0_deepbiome/.//weight/weight_257.h5 
[root    |INFO|build_network.py:84] Load trained model weight at ./simulation_s0/simulation_s0_deepbiome/.//weight/weight_258.h5 
[root    |INFO|build_network.py:84] Load trained model weight at ./simulation_s0/simulation_s0_deepbiome/.//weight/weight_259.h5 
[root    |INFO|build_network.py:84] Load trained model weight at ./simulation_s0/simulation_s0_deepbiome/.//weight/weight_260.h5 
[root    |INFO|build_network.py:84] Load trained model weight at ./simulation_s0/simulatio

...

[root    |INFO|build_network.py:84] Load trained model weight at ./simulation_s0/simulation_s0_deepbiome/.//weight/weight_702.h5 
[root    |INFO|build_network.py:84] Load trained model weight at ./simulation_s0/simulation_s0_deepbiome/.//weight/weight_703.h5 
[root    |INFO|build_network.py:84] Load trained model weight at ./simulation_s0/simulation_s0_deepbiome/.//weight/weight_704.h5 
[root    |INFO|build_network.py:84] Load trained model weight at ./simulation_s0/simulation_s0_deepbiome/.//weight/weight_705.h5 
[root    |INFO|build_network.py:84] Load trained model weight at ./simulation_s0/simulation_s0_deepbiome/.//weight/weight_706.h5 
[root    |INFO|build_network.py:84] Load trained model weight at ./simulation_s0/simulation_s0_deepbiome/.//weight/weight_707.h5 
[root    |INFO|build_network.py:84] Load trained model weight at ./simulation_s0/simulation_s0_deepbiome/.//weight/weight_708.h5 
[root    |INFO|build_network.py:84] Load trained model weight at ./simulation_s0/simulatio

[root    |INFO|build_network.py:84] Load trained model weight at ./simulation_s0/simulation_s0_deepbiome/.//weight/weight_766.h5 
[root    |INFO|build_network.py:84] Load trained model weight at ./simulation_s0/simulation_s0_deepbiome/.//weight/weight_767.h5 
[root    |INFO|build_network.py:84] Load trained model weight at ./simulation_s0/simulation_s0_deepbiome/.//weight/weight_768.h5 
[root    |INFO|build_network.py:84] Load trained model weight at ./simulation_s0/simulation_s0_deepbiome/.//weight/weight_769.h5 
[root    |INFO|build_network.py:84] Load trained model weight at ./simulation_s0/simulation_s0_deepbiome/.//weight/weight_770.h5 
[root    |INFO|build_network.py:84] Load trained model weight at ./simulation_s0/simulation_s0_deepbiome/.//weight/weight_771.h5 
[root    |INFO|build_network.py:84] Load trained model weight at ./simulation_s0/simulation_s0_deepbiome/.//weight/weight_772.h5 
[root    |INFO|build_network.py:84] Load trained model weight at ./simulation_s0/simulatio

[root    |INFO|build_network.py:84] Load trained model weight at ./simulation_s0/simulation_s0_deepbiome/.//weight/weight_830.h5 
[root    |INFO|build_network.py:84] Load trained model weight at ./simulation_s0/simulation_s0_deepbiome/.//weight/weight_831.h5 
[root    |INFO|build_network.py:84] Load trained model weight at ./simulation_s0/simulation_s0_deepbiome/.//weight/weight_832.h5 
[root    |INFO|build_network.py:84] Load trained model weight at ./simulation_s0/simulation_s0_deepbiome/.//weight/weight_833.h5 
[root    |INFO|build_network.py:84] Load trained model weight at ./simulation_s0/simulation_s0_deepbiome/.//weight/weight_834.h5 
[root    |INFO|build_network.py:84] Load trained model weight at ./simulation_s0/simulation_s0_deepbiome/.//weight/weight_835.h5 
[root    |INFO|build_network.py:84] Load trained model weight at ./simulation_s0/simulation_s0_deepbiome/.//weight/weight_836.h5 
[root    |INFO|build_network.py:84] Load trained model weight at ./simulation_s0/simulatio

[root    |INFO|build_network.py:84] Load trained model weight at ./simulation_s0/simulation_s0_deepbiome/.//weight/weight_894.h5 
[root    |INFO|build_network.py:84] Load trained model weight at ./simulation_s0/simulation_s0_deepbiome/.//weight/weight_895.h5 
[root    |INFO|build_network.py:84] Load trained model weight at ./simulation_s0/simulation_s0_deepbiome/.//weight/weight_896.h5 
[root    |INFO|build_network.py:84] Load trained model weight at ./simulation_s0/simulation_s0_deepbiome/.//weight/weight_897.h5 
[root    |INFO|build_network.py:84] Load trained model weight at ./simulation_s0/simulation_s0_deepbiome/.//weight/weight_898.h5 
[root    |INFO|build_network.py:84] Load trained model weight at ./simulation_s0/simulation_s0_deepbiome/.//weight/weight_899.h5 
[root    |INFO|build_network.py:84] Load trained model weight at ./simulation_s0/simulation_s0_deepbiome/.//weight/weight_900.h5 
[root    |INFO|build_network.py:84] Load trained model weight at ./simulation_s0/simulatio

[root    |INFO|build_network.py:84] Load trained model weight at ./simulation_s0/simulation_s0_deepbiome/.//weight/weight_958.h5 
[root    |INFO|build_network.py:84] Load trained model weight at ./simulation_s0/simulation_s0_deepbiome/.//weight/weight_959.h5 
[root    |INFO|build_network.py:84] Load trained model weight at ./simulation_s0/simulation_s0_deepbiome/.//weight/weight_960.h5 
[root    |INFO|build_network.py:84] Load trained model weight at ./simulation_s0/simulation_s0_deepbiome/.//weight/weight_961.h5 
[root    |INFO|build_network.py:84] Load trained model weight at ./simulation_s0/simulation_s0_deepbiome/.//weight/weight_962.h5 
[root    |INFO|build_network.py:84] Load trained model weight at ./simulation_s0/simulation_s0_deepbiome/.//weight/weight_963.h5 
[root    |INFO|build_network.py:84] Load trained model weight at ./simulation_s0/simulation_s0_deepbiome/.//weight/weight_964.h5 
[root    |INFO|build_network.py:84] Load trained model weight at ./simulation_s0/simulatio

In [18]:
summary

,Model,PhyloTree,No. true taxa,No. total taxa,Sensitivity_mean,Sensitivity_std,Specificity_mean,Specificity_std,Gmeasure_mean,Gmeasure_std,Accuracy_mean,Accuracy_std
0,DeepBiome,Genus,31,48,0.954452,0.041715,0.668648,0.086583,0.796819,0.053031,0.673263,0.085101
1,,Family,23,40,0.966739,0.036733,0.828151,0.061709,0.893913,0.036529,0.831615,0.060096
2,,Order,9,23,0.970000,0.057639,0.854914,0.057012,0.909717,0.042093,0.857563,0.055752
3,,Class,7,17,0.983429,0.050009,0.835123,0.063344,0.905246,0.042543,0.841908,0.060487


In [19]:
print('%7s & %7s & %12s & %s' % ('Model', 'PhyloTree', 'True (Total)', ' & '.join(summary.columns[4:])))
print('---------------------------------------------------------------------------------------------------------------')
for i in range(summary.shape[0]):
    print('%10s & %7s & %7d (%d) & ' % tuple(summary.iloc[i,:4]) + ' &'.join(['%6.3f' % val for val in summary.iloc[i,4:]]) + ' \\\\')
    
# if save: 
#     # filenametexa = '.'.join(["%s_select_texa_1" % filename.split('.')[0], filename.split('.')[1]])
#     colname = ['Tree','True (Total)','Selected','Sensitivity','Specificity','gMeasure','Accuracy']
#     with open('%s/%s' % (analysis_dir, filename), mode='a') as f:
#     #     f.write('---\ntitle: "%s texa selection ver.1"\noutput: html_document\n---\n\n' % filename.split('.')[0])
#         f.write('\n## Texa Selection Preformance (ver 1): %s\n\n' % model_aka)
#         f.write('| %s |\n' % ('|'.join([v for v in colname])))
#         f.write('|'+'---|'*len(colname)+'\n')
#         for value in values:
#             f.write('| %s |\n' % ('|'.join(value)))

  Model & PhyloTree & True (Total) & Sensitivity_mean & Sensitivity_std & Specificity_mean & Specificity_std & Gmeasure_mean & Gmeasure_std & Accuracy_mean & Accuracy_std
---------------------------------------------------------------------------------------------------------------
 DeepBiome &   Genus &      31 (48) &  0.954 & 0.042 & 0.669 & 0.087 & 0.797 & 0.053 & 0.673 & 0.085 \\
           &  Family &      23 (40) &  0.967 & 0.037 & 0.828 & 0.062 & 0.894 & 0.037 & 0.832 & 0.060 \\
           &   Order &       9 (23) &  0.970 & 0.058 & 0.855 & 0.057 & 0.910 & 0.042 & 0.858 & 0.056 \\
           &   Class &       7 (17) &  0.983 & 0.050 & 0.835 & 0.063 & 0.905 & 0.043 & 0.842 & 0.060 \\
